A simple demo to use the pre-trained model for denoising an image

In [2]:
import os
import sys
import numpy as np
import skimage.io
import matplotlib.pyplot as plt
import cv2
# fixed seed
np.random.seed(2019)

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import BMCNN
sys.path.append(ROOT_DIR)
from bmcnn import utils
import bmcnn.model as nmodel

#loading denoiser model
denoiser = nmodel.ImageDenoiser(graph = 'bmcnn', fs = 64, model_weights = os.path.abspath("../savedmodels/model_ver2.0.h5"))


Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [3]:
# Visualize before and after images
def visualize_image(img_n, im_den, psnr_n, psnr_d):
    """
    img_n: noisy image
    im_den: denoised image
    psnr_n: pnsr of noisy image
    psnr_d : pnsr of denoised image
    """
    plt.figure(figsize=(14, 14))
    plt.subplot(1, 2, 1)
    plt.title("Noisy image PSNR = %2.2f dB"%psnr_n, fontsize=10)
    plt.axis('off')
    if img_n.ndim == 2:
        if np.max(img_n)>1:
            img_n = img_n/255.0
        plt.imshow(skimage.color.gray2rgb(img_n))
    else:
        plt.imshow(img_n)
    
    plt.subplot(1, 2, 2)
    plt.title("Denoised image PSNR = %2.2f dB"%psnr_d, fontsize=10)
    plt.axis('off')
    if img_n.ndim == 2:
        if np.max(img_n)>1:
            im_den = im_den/255.0
        plt.imshow(skimage.color.gray2rgb(im_den))
    else:
        plt.imshow(im_den)

    plt.show()


## Sample 1: adding noise and denoising a color image 

In [4]:
reference_image = '../testdata/fruits.png'


sigma = 8
img_ref = skimage.io.imread(reference_image)
img_n = utils.addNoise(img_ref, sigma)

im_out = denoiser.run(img_n, sigma)

_, psnr_n =  utils.mse_psnr(img_ref, img_n)
_, psnr_d =  utils.mse_psnr(img_ref, im_out)

visualize_image(img_n, im_out, psnr_n, psnr_d)

skimage.io.imsave('fruits_out.png', np.uint8(im_out*255+0.5))



ResourceExhaustedError: OOM when allocating tensor with shape[4,258,258,64] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node conv2d_2/convolution/SpaceToBatchND}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node conv2d_7/BiasAdd}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


## Sample 2: adding noise and denoising a grayscale image 

In [ ]:
reference_image = '../testdata/cameraman.png'

sigma = 20
img_ref = skimage.io.imread(reference_image)
img_n = utils.addNoise(img_ref, sigma)


im_out = denoiser.run(img_n, sigma)

_, psnr_n =  utils.mse_psnr(img_ref, img_n)
_, psnr_d =  utils.mse_psnr(img_ref, im_out)

visualize_image(img_n, im_out, psnr_n, psnr_d)

skimage.io.imsave('cameraman_out.png', np.uint8(im_out*255+0.5))



In [ ]:
def increase_contrast(img):
    #-----Converting image to LAB Color model----------------------------------- 
    lab= cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

    #-----Splitting the LAB image to different channels-------------------------
    l, a, b = cv2.split(lab)

    #-----Applying CLAHE to L-channel-------------------------------------------
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    cl = clahe.apply(l)

    #-----Merge the CLAHE enhanced L-channel with the a and b channel-----------
    limg = cv2.merge((cl,a,b))

    #-----Converting image from LAB Color model to RGB model--------------------
    final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    return final

In [ ]:
reference_image = './img5.png'
img_n = cv2.imread(reference_image)
img_n = cv2.resize(img_n, (1280,552))
print(img_n.shape)
#img_n = increase_contrast(img_n)
plt.imshow(img_n)
plt.show()

sigma = 20

im_out = denoiser.run(img_n, sigma)
print(im_out.shape)
im_out=np.array((im_out*255.0), dtype='uint8')
im_out = increase_contrast(im_out)
skimage.io.imsave('im_out.png', im_out)
plt.imshow(im_out)
plt.show()

## Sample 3: denoising a color image 

In [ ]:
reference_image = '../testdata/lena_color.png'
noisy_image = '../testdata/lena_noisy_s12.png'

sigma = 12
img_ref = skimage.io.imread(reference_image)
img_n = skimage.io.imread(noisy_image)

im_out = denoiser.run(img_n, sigma)

_, psnr_n =  utils.mse_psnr(img_ref, img_n)
_, psnr_d =  utils.mse_psnr(img_ref, im_out)

visualize_image(img_n, im_out, psnr_n, psnr_d)

skimage.io.imsave('lena_out.png', np.uint8(im_out*255+0.5))


## Sample 4: denoising a grayscale image 


In [ ]:
reference_image = '../testdata/foreman.png'
noisy_image = '../testdata/foreman_noisy_s15.png'

sigma = 15
img_ref = skimage.io.imread(reference_image)
img_n = skimage.io.imread(noisy_image)

im_out = denoiser.run(img_n, sigma)

_, psnr_n =  utils.mse_psnr(img_ref, img_n)
_, psnr_d =  utils.mse_psnr(img_ref, im_out)

visualize_image(img_n, im_out, psnr_n, psnr_d)

skimage.io.imsave('foreman_out.png', np.uint8(im_out*255+0.5))
